## Implement LMS, NLMS and RLS filters and compare performance for noise filtering

In [1]:
import numpy as np
from scipy.io.wavfile import read, write
import time


In [2]:
# load data and normalize
fs1, d = read('data/Noise.wav')
d = d/max(d)
# Signal
fs1, x = read('data/Music.wav')
x = x/max(x)
# Noisy signal
Fs, u = read('data/Noisy_Music.wav')
u_max = max(u)
u = u/u_max

In [14]:
def calc_power(signal):
    return np.mean(np.abs(signal)**2)

def calc_SNR(signal, noise):
    P_sig = calc_power(signal)
    P_noise = calc_power(noise)
    return 10*np.log10(P_sig/P_noise)

In [3]:
# setup filter parameters
N = len(d)
M = 12

# to find the appropriate step size, we need to estimate the power of the signal
P_u = 1/N * np.sum(u**2)

# mu is found with (mu = alpha / P_u), where alpha is a small constant between 0.01 and 0.1. Stability is guaranteed if mu < 2 / P_u
mu = 0.01 / P_u

In [4]:
# implement LMS
LMS_start = time.time()
w_lms = np.random.randn(M) # initialize filter weights at 0
u_pad = np.hstack((np.zeros(M-1), u)) # pad noisy signal with zeros
y = np.zeros(N) # initialize output signal

for n in range(0, N):
    u_n = u_pad[n:n+M] # grab M amount of samples from observed signal (-1 because we want to go backwards)
    e = d[n] - np.matmul(w_lms,u_n) # calculate error
    w_lms = w_lms + mu*e*u_n # update weights
    y[n] = np.matmul(w_lms, u_n) # apply weights to observed signal
LMS_done = time.time()
    
# calculate and save output signal
LMS_output = u - y
write(f"filtered_signal_LMS_16b.wav", Fs, (LMS_output*u_max).astype(np.int16))

#evaluate LMS with time and SNR
LMS_time = LMS_done - LMS_start
SNR_before = calc_SNR(x, u-x)
print(f"SNR before filtering: {SNR_before:.2f} dB")

